<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

# <font color='blue'>Data Science Academy</font>

# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Projeto 5</font>

## <font color='blue'>Construindo Chatbot Inteligente com LLM Open Source Falcon</font>

In [19]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [20]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark
import warnings
warnings.filterwarnings('ignore')

In [21]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q accelerate einops xformers
!pip install -q -U optimum

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
xformers 0.0.22.post7 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.


In [22]:
!pip install -q transformers==4.34

In [23]:
# Imports
import torch
import transformers
import bitsandbytes
import optimum
import safetensors
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [24]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

transformers: 4.34.0
bitsandbytes: 0.41.2.post2
safetensors : 0.4.1
optimum     : 1.14.1
torch       : 2.1.0



In [25]:
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: Tesla V100-SXM2-16GB
Total Memória [GB] da GPU: 16.935419904


## Definindo os Parâmetros de Quantização

Os parâmetros abaixo são configurações específicas para a quantização de modelos de aprendizado de máquina com o PyTorch. Aqui está uma explicação para cada um deles:

**load_in_4bit (True):** Este parâmetro indica que o modelo deve ser carregado em um formato de 4 bits. Isso é  definido para reduzir o uso de memória, permitindo que modelos grandes sejam carregados em hardware com memória limitada. Quando True, o modelo é carregado com uma precisão reduzida.

**bnb_4bit_compute_dtype (torch.float16):** Define o tipo de dados a ser usado para cálculos internos quando o modelo está em formato de 4 bits. O torch.float16 indica que os cálculos serão realizados usando o tipo de dados de ponto flutuante de 16 bits, que é uma boa combinação de precisão e eficiência para muitos modelos de aprendizado de máquina.

**bnb_4bit_quant_type ("nf4"):** Especifica o tipo de quantização a ser aplicada. O "nf4" se refere a um tipo específico de quantização de 4 bits. Referência aqui: https://huggingface.co/blog/4bit-transformers-bitsandbytes

**bnb_4bit_use_double_quant (True):** Indica se deve ser usada uma "quantização dupla" durante o processo de quantização. A quantização dupla pode ajudar a manter mais precisão nos dados quantizados, mas pode ter um custo computacional mais alto.

**llm_int8_enable_fp32_cpu_offload (True):** Este parâmetro sugere que para operações de baixa latência (em modelos grandes como LLMs - Large Language Models), quando os cálculos estão sendo feitos em int8 (8 bits), há uma opção para descarregar (offload) algumas operações para o CPU em formato fp32 (ponto flutuante de 32 bits). Isso pode ser útil para equilibrar carga entre CPU e GPU e para lidar com operações que exigem maior precisão. Referência aqui: https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Esses parâmetros são avançados e específicos para otimizações de desempenho e uso de memória em modelos de aprendizado de máquina, especialmente modelos grandes que podem ter restrições de hardware.

In [26]:
# Define os parâmetros
quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                         bnb_4bit_compute_dtype = torch.float16,
                                         bnb_4bit_quant_type = "nf4",
                                         bnb_4bit_use_double_quant = True,
                                         llm_int8_enable_fp32_cpu_offload = True)

## Carrega o Modelo

https://huggingface.co/tiiuae/falcon-7b

In [27]:
# Define o modelo
model_name = "tiiuae/falcon-7b-instruct"

In [28]:
# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [29]:
# Carrega o modelo
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map = "auto",
                                             quantization_config = quantization_config,
                                             trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Instruindo o LLM Diretamente

In [30]:
# Cria o pipeline
pipeline = transformers.pipeline("text-generation",

    # Especifica o modelo pré-treinado a ser utilizado para a geração de texto.
    model = model,

    # Define o tokenizer a ser usado para converter texto em tokens.
    tokenizer = tokenizer,

     # Atribui automaticamente o modelo a dispositivos disponíveis (GPUs/CPUs) para otimizar o desempenho.
    device_map = "auto",

    # Define o comprimento máximo do texto gerado (número de tokens).
    max_length = 200,

    # Permite o uso de cache para acelerar a geração repetindo cálculos anteriores.
    use_cache = True,

    # Ativa a amostragem durante a geração de texto, permitindo maior variação e criatividade nos resultados.
    do_sample = True,

    # Limita a escolha do próximo token aos 'k' mais prováveis, neste caso 10,
    # para balancear criatividade e coesão.
    top_k = 10,

    # Define o número de sequências de texto distintas a serem geradas em uma única execução.
    num_return_sequences = 1,

    # Define o token que indica o fim de uma sequência (End Of Sequence).
    eos_token_id = tokenizer.eos_token_id,

    # Define o token usado para preenchimento (padding), geralmente igual ao EOS em modelos como GPT.
    pad_token_id = tokenizer.eos_token_id)

## Conversando com o Chatbot em Português

In [31]:
# Envia mensagem para o modelo
mensagem1 = pipeline("Crie uma lista de quatro itens importantes que um piloto de avião deve observar")

In [32]:
# Resposta do modelo
for seq in mensagem1:
    print(f"\nResposta1: {seq['generated_text']}")


Resposta1: Crie uma lista de quatro itens importantes que um piloto de avião deve observar
1. Atençãoção constante aos pilotos durante o voo, a fim de identificar qualquer alteração no avião.
2. Estudo do plano de voo e execução de todas as manobras e comandos.
3. Atençãoção ao equipamento e a bordo, no meio do voo.
4. Manutenção do equipamento e acompanhamento do consumo de combustível.


In [33]:
# Envia mensagem para o modelo
mensagem2 = pipeline("Liste cinco sinais precoces de gravidez em mulheres")

In [34]:
# Resposta do modelo
for seq in mensagem2:
    print(f"\nResposta2: {seq['generated_text']}")


Resposta2: Liste cinco sinais precoces de gravidez em mulheres
Ora, em mulheres, gravidez é mais antiga que as sementeis. Para quem está a tentar conciliar as duas, a lista de sinais precoces não se arranque na sua lista de gravidez. A lista segue abaixo. Sinales do período anterior ao gravidez.
1. Aumento no volume do ventre – o aumento é um signo positivo de gravidez que a primeira vez é o momento que a mulher é realmente em a gravidez! Sinalo de “gravidez” que a primeira vez foi depois de a primeira vez.
2. Perda de apetite – o apetite foi uma coisa que eu nunca senti na primeira vez que eu tinha gravidez (eu só com a segunda gravidez).
3. Duraço da primeira gravidade – eu


In [35]:
# Envia mensagem para o modelo
mensagem3 = pipeline("Liste 3 tecnologias para quem deseja trabalhar com Inteligência Artificial")

In [36]:
# Resposta do modelo
for seq in mensagem3:
    print(f"Resposta3: {seq['generated_text']}")

Resposta3: Liste 3 tecnologias para quem deseja trabalhar com Inteligência Artificial.
1. Machine Learning
2. Natural Language Processing
3. Computer Vision


# Fim